# 手寫數字辨識
Author: WeiHong Lee

In [ ]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST/MNIST_data/", one_hot=True)
print('train image:', mnist.train.images.shape)
print('train label:', mnist.train.labels.shape)
print('test image:', mnist.test.images.shape)
print('test label:', mnist.test.labels.shape)

In [ ]:
# Hyper-Parameters
learning_rate = 0.001
batch_size = 100
total_step = 3000

# Network Parameters
n_input = 784
n_hidden_1 = 512
n_hidden_2 = 512
n_classes = 10

In [ ]:
x = tf.placeholder(tf.float32, [None, n_input])
y_ = tf.placeholder(tf.float32, [None, n_classes])

with tf.name_scope('hidden1'):
    weights = tf.Variable(tf.random_normal([n_input, n_hidden_1]),
                          name='weights')
    biases = tf.Variable(tf.zeros([n_hidden_1]),
                         name='biases')
    layer_1 = tf.nn.relu(tf.matmul(x, weights) + biases)
    
with tf.name_scope('hidden2'):
    weights = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),
                          name='weights')
    biases = tf.Variable(tf.zeros([n_hidden_2]),
                         name='biases')
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights) + biases)

with tf.name_scope('output'):
    weights = tf.Variable(tf.random_normal([n_hidden_2, n_classes]),
                          name='weights')
    biases = tf.Variable(tf.zeros([n_classes]),
                         name='biases')
    logits = tf.matmul(layer_2, weights) + biases

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
tf.summary.scalar('loss', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    y = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
tf.summary.scalar('acc', accuracy)

In [ ]:
init = tf.global_variables_initializer()
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    
    train_writer = tf.summary.FileWriter('MNIST/logs/train_relu', sess.graph)
    test_writer = tf.summary.FileWriter('MNIST/logs/test_relu')
    
    for i in range(total_step):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y_: batch_ys})
        train_writer.add_summary(summary, i)
        
        if i % 10 == 0:
            summary = sess.run(merged, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
            test_writer.add_summary(summary, i)
        
        if i % 100 == 0:
            loss, acc = sess.run([cross_entropy, accuracy],
                                 feed_dict={x: batch_xs, y_: batch_ys})
            test_loss, test_acc = sess.run([cross_entropy, accuracy], 
                                           feed_dict={x: mnist.test.images, y_: mnist.test.labels})
            print('step: {:4d}\t loss: {:.2f}\t acc: {:.2f}\t test_loss: {:.2f}\t test_acc: {:.2f}'.
                  format(i, loss, acc, test_loss, test_acc))

    train_writer.close()
    test_writer.close()